# Effects em TypeScript - Uma abordagem funcional

Este notebook explora o conceito de **Effects** em programação funcional usando TypeScript. 

## O que são Effects?

Um `Effect` é essencialmente um **thunk** - uma função que não recebe argumentos e retorna um valor quando chamada. Esta é uma abstração poderosa que nos permite separar a **descrição** de uma computação da sua **execução**.

### Por que isso é importante?

Em programação imperativa tradicional, quando escrevemos:
```typescript
const result = expensiveOperation();
```

A operação é executada **imediatamente**. Com Effects, podemos descrever a operação sem executá-la:
```typescript
const effect = () => expensiveOperation();  // Descrição da operação
const result = effect();  // Execução quando necessário
```

### Vantagens dos Effects:

1. **Controle de Execução**: Decidimos **quando** e **quantas vezes** executar
2. **Composição**: Podemos combinar Effects de forma elegante
3. **Testabilidade**: Easier testing através de injeção de dependência
4. **Lazy Evaluation**: Computações só acontecem quando necessário
5. **Error Handling**: Tratamento de erros mais expressivo e controlado

Isso nos permite construir sistemas mais robustos, testáveis e compostos através de abstrações funcionais poderosas.

In [27]:
type Effect<A> = () => A;

In [28]:
// Isso é uma função normal, ela retorna um number
function randomNumber(): number {
    return Math.floor(Math.random() * 100) + 1;
}

// Essa é uma função que retorna um `Effect` de number
// Ou seja, ela não executa a função imediatamente, mas retorna uma função que, 
// quando chamada, executará a função original e retornará o resultado.
function randomNumberFunction(): Effect<number> {
    return randomNumber;
}

// Mesma coisa que
// const randomNumberFunction = (): Effect<number> => () => randomNumber();

// Vamos criar um helper que simplesmente roda o `Effect` e imprime o resultado
function run<A>(effect: Effect<A>): void {
    console.log(effect());
}

run(randomNumberFunction());

2


## Por que usar Effects?

**Porque agora temos uma função que pode ser executada mais tarde!**

### O Poder da Composição

Em vez de modificar o resultado diretamente, conseguimos modificar a **função em si**. Isso abre possibilidades incríveis:

1. **Repetição Controlada**: Podemos executar uma operação múltiplas vezes
2. **Transformação de Comportamento**: Adicionar funcionalidades como retry, timeout, logging
3. **Composição Declarativa**: Construir operações complexas a partir de simples

### Exemplo Prático: Repeat

Note que `repeat` não executa a função imediatamente - ela retorna um **novo Effect** que, quando executado, retorna uma lista de resultados. Isso é fundamental: Effects sempre retornam Effects, permitindo composição infinita.

### Lazy Evaluation em Ação

```typescript
// Sem Effects - execução imediata
const numbers = Array.from({length: 5}, () => randomNumber());  // Executa AGORA

// Com Effects - execução controlada  
const repeatEffect = repeat(randomNumberFunction(), 5);  // Apenas descrição
const numbers = repeatEffect();  // Executa QUANDO quisermos
```

Esta separação entre descrição e execução é a essência dos Effects e permite que construamos sistemas mais flexíveis e controlados.

In [29]:
function repeat<A>(effect: Effect<A>, times: number): Effect<A[]> {
    return () => {
        const results: A[] = [];
        for (let i = 0; i < times; i++) {
            const result = effect();
            results.push(result);
        }
        return results;
    };
}

// Agora podemos fazer composição de efeitos
// Podemos repetir a execução de `randomNumberFunction` 5 vezes
run(repeat(randomNumberFunction(), 5));

[ 71, 12, 39, 97, 48 ]


## A Magia da Lazy Evaluation

Note que até eu fazer `run`, **nenhuma computação acontece**. Estamos apenas construindo uma "receita" de como executar as operações.

### Reutilização e Flexibilidade

Como `repeat` retorna um novo `Effect`, posso:
- **Reutilizar** a mesma "receita" múltiplas vezes
- **Compor** com outras operações
- **Modificar** o comportamento sem alterar o código original

### Benefícios Práticos:

1. **Performance**: Computações caras só acontecem quando necessário
2. **Debugging**: Posso inspecionar a "receita" antes de executar
3. **Testing**: Posso substituir Effects por mocks facilmente
4. **Caching**: Posso implementar cache ao nível do Effect

Isso é especialmente poderoso quando lidamos com operações que podem falhar, como chamadas de rede, I/O ou operações custosas.

In [30]:
const twice = repeat(randomNumberFunction(), 2);
const thrice = repeat(randomNumberFunction(), 3);
const twiceTwice = repeat(twice, 2);

run(twice);
run(thrice);
run(twiceTwice);

[ 71, 34 ]
[ 34, 6, 15 ]
[ [ 80, 84 ], [ 33, 24 ] ]


## Lidando com Erros de Forma Funcional

O tratamento de erros em programação funcional é fundamentalmente diferente da abordagem imperativa. Em vez de usar try-catch em todo lugar, construímos Effects que **encapsulam** a possibilidade de falha.

### Filosofia dos Effects com Erros:

1. **Erros são Valores**: Tratamos falhas como parte normal do fluxo
2. **Composição Segura**: Effects podem ser compostos mesmo quando podem falhar
3. **Controle Explícito**: Decidimos onde e como tratar erros

### Benefícios desta Abordagem:

- **Previsibilidade**: Sabemos exatamente onde erros podem ocorrer
- **Composição**: Podemos adicionar retry, fallbacks, logging de forma declarativa
- **Separação de Responsabilidades**: Lógica de negócio separada do tratamento de erros

**Lembre-se:** o `thunk` (nossa função Effect) não possui argumentos, mas uma função pode receber argumentos e retornar um `Effect` que **captura** esses argumentos no closure.

### Exemplo Prático:
```typescript
// Em vez de:
try {
    const result = riskyOperation(param);
    process(result);
} catch (error) {
    handleError();
}

// Usamos:
const effect = createRiskyEffect(param);
const safeEffect = addErrorHandling(effect);
const result = safeEffect();
```

In [31]:
function failIfSmallNumber(numberEffect: Effect<number>): Effect<number | Error> {
    return () => {
        const n = numberEffect();
        if (n < 70) {
            return new Error("Number is too small");
        }
        return n;
    };
}

// Não precisa de try/catch
run(failIfSmallNumber(randomNumberFunction()));

Error: Number is too small
    at <anonymous>:5:14
    at run (<anonymous>:15:15)
    at <anonymous>:11:1


## Padrão Retry - Construindo Resiliência

O erro só acontece **DEPOIS** de chamar `run`. Isso é fundamental! Conseguimos construir toda a lógica de retry **antes** de qualquer execução acontecer.

### Por que Retry é Poderoso com Effects?

1. **Composição Natural**: Retry é apenas outro Effect que wrapa outros Effects
2. **Configuração Declarativa**: Podemos definir estratégias complexas de retry
3. **Transparência**: O Effect resultante ainda é um Effect normal

### Estratégias de Retry Avançadas:

Com Effects, podemos facilmente implementar:
- **Exponential Backoff**: Aumentar tempo entre tentativas
- **Circuit Breaker**: Parar tentativas após muitas falhas
- **Conditional Retry**: Retry apenas para certos tipos de erro
- **Jittered Retry**: Adicionar randomness para evitar thundering herd

Vamos criar um efeito que tenta executar um efeito várias vezes até ter sucesso:

In [32]:
function retry<A>(effect: Effect<A>, times: number): Effect<A | Error> {
    return () => {
        for (let i = 0; i < times; i++) {
            const result = effect();
            if (result instanceof Error) {
                console.log("Effect failed, retrying...");
                continue;
            }
            return result;
        }
        return new Error("Effect failed after retries");
    };
}

// Vamos tentar executar o efeito que falha 3 vezes
run(retry(failIfSmallNumber(randomNumberFunction()), 3));

Effect failed, retrying...
98


### Observando o Comportamento do Retry

Execute a célula acima algumas vezes e observe como o retry funciona na prática. Você verá:

1. **Tentativas Múltiplas**: O Effect tenta até 3 vezes antes de desistir
2. **Feedback Visual**: Cada falha é logada, mostrando o processo
3. **Composição Transparente**: O resultado final ainda é um Effect simples

Este padrão é extremamente comum em sistemas distribuídos e APIs, onde falhas temporárias são esperadas.

## Padrão OrElse - Graceful Degradation

Mas as vezes o erro não é o fim do mundo, e tem um valor que podemos utilizar.

O padrão `orElse` implementa **graceful degradation** - quando algo falha, fornecemos um valor padrão sensato em vez de quebrar todo o sistema.

### Filosofia do Graceful Degradation:

1. **Sistemas Resilientes**: Preferem funcionar parcialmente a não funcionar
2. **Experiência do Usuário**: Melhor mostrar dados parciais que uma tela de erro
3. **Disponibilidade**: Mantém o sistema funcionando mesmo com falhas

### Estratégias de Fallback:

- **Valor Padrão**: Como mostrado no exemplo (retorna 0)
- **Cache**: Usar dados antigos quando novos não estão disponíveis  
- **Serviço Alternativo**: Tentar outro endpoint/serviço
- **Dados Estáticos**: Usar configurações fixas como último recurso

### Composição de Padrões de Resiliência:

**SEMPRE, SEMPRE** retornamos um `Effect`. Isso permite que todas as funções sejam compostas e encadeadas, criando pipelines de resiliência:

```
Original Effect -> Retry -> OrElse -> Telemetry -> Execute
```

Cada etapa adiciona uma camada de robustez sem quebrar a composição.

In [33]:
function orElse<A>(effect: Effect<A>, fallback: A): Effect<A> {
    return () => {
        const result = effect();
        if (result instanceof Error) {
            console.log("Effect failed, returning fallback value:", fallback);
            return fallback;
        }
        console.log("Effect succeeded with value:", result);
        return result;
    };
}

// Se falhar mesmo depois de tentar, vira 0
const retryIfSmall = retry(failIfSmallNumber(randomNumberFunction()), 2);
run(repeat(orElse(retryIfSmall, 0), 20));

Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 71
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 73
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect succeeded with value: 83
Effect failed, retrying...
Effect succeeded with value: 94
Effect succeeded with value: 73
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect suc

## Telemetria e Observabilidade - Monitoramento Transparente

O legal é que como temos controle de **QUANDO** executar o efeito, podemos adicionar funcionalidades de monitoramento **antes**, **durante** e **depois** da execução, sem modificar a lógica original.

### Observabilidade em Sistemas Modernos:

1. **Logs**: Registro de eventos para debugging
2. **Métricas**: Medições quantitativas (latência, throughput, erro rate)
3. **Tracing**: Acompanhamento de requisições através do sistema
4. **Alerting**: Notificações quando algo está errado

### Vantagens dos Effects para Observabilidade:

- **Transparência**: Adicionar telemetria sem modificar código de negócio
- **Composição**: Combinar múltiplos tipos de observabilidade
- **Configuração**: Facilmente ligar/desligar em diferentes ambientes
- **Testing**: Pode ser mockado ou desabilitado em testes

### O Princípio da Transparência

**Fundamental:** Funções de observabilidade como `logEffect` são **transparentes** - elas **observam** o valor mas **não o modificam**. Isso significa:

```typescript
// O resultado é EXATAMENTE o mesmo
const originalEffect = randomNumberFunction();
const loggedEffect = logEffect(originalEffect, "debug");

// Ambos retornam o mesmo valor, mas loggedEffect adiciona logs
```

**Por que isso é importante:**
- **Comportamento Preservado**: A lógica de negócio permanece intacta
- **Composição Segura**: Pode ser adicionado/removido sem quebrar o código
- **Testing**: Logs podem ser facilmente desabilitados em testes
- **Performance**: Zero overhead quando logging está desabilitado

Cada wrapper adiciona uma camada de observabilidade mantendo a interface Effect consistente.

In [34]:
function logEffect<A>(effect: Effect<A>, text: string): Effect<A> {
    return () => {
        console.log(`[--> Starting effect: ${text}`);
        const result = effect();
        console.log(`[--> End effect: ${text}`);
        return result;
    };
}

const operation = orElse(retry(failIfSmallNumber(randomNumberFunction()), 2), 0);

run(repeat(logEffect(operation, "Repeat Effect"), 10));

[--> Starting effect: Repeat Effect
Effect succeeded with value: 82
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect succeeded with value: 88
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect succeeded with value: 97
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect succeeded with value: 72
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect succeeded with value: 82
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: Repeat Effect
[--> Starting effect: Repeat Effect
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> End effect: 

## Transformação de Dados com Effects

O Effect não precisa sempre retornar o mesmo tipo `A` - podemos **transformar** o resultado durante a execução.

### Exemplo: Medição de Performance

Aqui criamos um Effect que retorna não apenas o resultado original, mas **também** quanto tempo a operação demorou. Isso é útil para:

1. **Performance Monitoring**: Detectar operações lentas
2. **SLA Tracking**: Verificar se estamos dentro dos tempos esperados  
3. **Optimization**: Identificar gargalos de performance
4. **Alerting**: Disparar alertas quando operações demoram muito

In [35]:
function timed<A>(effect: Effect<A>): Effect<[A, number]> {
    return () => {
        const start = performance.now();
        const result = effect();
        const end = performance.now();
        return [result, (end - start) * 10];
    };
}

const retryIfSmall2 = retry(failIfSmallNumber(randomNumberFunction()), 2);
const zeroIfSmall = orElse(retryIfSmall2, 0);
const timedOperation = timed(zeroIfSmall);

run(repeat(timedOperation, 5));

Effect succeeded with value: 93
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
[
  [ 93, 0.8442400000058115 ],
  [ 0, 0.8908499998506159 ],
  [ 0, 0.4678899998543784 ],
  [ 0, 0.1686199998948723 ],
  [ 0, 1.1151799996150658 ]
]


## O Problema dos Parênteses Aninhados

Vai tomar no cu, tem tanto parêntesis que eu fiquei perdido!

### O Problema da Composição Nested

Quando composição de funções fica complexa, temos vários problemas:

1. **Legibilidade**: Difícil de ler e entender o fluxo
2. **Manutenibilidade**: Difícil de adicionar/remover steps
3. **Debugging**: Complicado identificar onde erros acontecem
4. **Ordem de Execução**: Precisa ler de dentro para fora

### Composição Funcional vs Procedural

```typescript
// Estilo aninhado - confuso
const result = h(g(f(data)));

// Estilo procedural - verboso  
// E dar nomes é um dos problemas mais difícieis na programação
const temp1 = f(data);
const temp2 = g(temp1);
const result = h(temp2);

// Estilo pipeline - claro
const result = pipe(data, f, g, h);
```

### A Inspiração UNIX

**Solução:** No UNIX a solução é encadear comandos com pipe!
- Exemplo: `ls -l | grep py | wc -l` em vez de `wc -l (grep py (ls -l))`
- Cada comando processa a saída do anterior
- Leitura da esquerda para direita (ordem natural)
- Fácil adicionar/remover steps no meio

### Pipeline em Programação Funcional

Bibliotecas como Ramda e `pipe-ts` têm uma função chamada `pipe` que replica essa funcionalidade:
- `pipe(data, f, g, h)` === `h(g(f(data)))`
- Leitura natural da esquerda para direita
- Cada função recebe o resultado da anterior
- Composição declarativa e clara

### Por que Precisamos de Funções Auxiliares no Pipe?

O `pipe` espera que cada função receba **exatamente um argumento** (o resultado da função anterior). Mas muitas das nossas funções como `retry`, `orElse` e `repeat` precisam de argumentos adicionais:

```typescript
// Problema: retry precisa de 2 argumentos
retry(effect, 2);  // Effect + número de tentativas

// Solução: função que "fixa" o segundo argumento
const withRetry = (times: number) => (effect: Effect<A>) => retry(effect, times);
```

**Por que isso funciona:**
1. **Aridade Unária**: Pipe exige funções que recebem 1 argumento
2. **Closure**: Função captura os argumentos extras (2, 0, 20, etc.)
3. **Adaptação**: Transformamos função de N argumentos em função de 1 argumento
4. **Composição**: Todas as etapas do pipeline têm a mesma interface

Sem essa adaptação, o pipe quebraria porque tentaria passar apenas o Effect para funções que esperam mais parâmetros.

In [36]:
// Usando pipe-ts: uma biblioteca especializada para composição funcional
import { pipe } from 'npm:pipe-ts';

const operation = pipe(
    randomNumberFunction,
    failIfSmallNumber, // Nesse caso, o primeiro argumento é o Effect, então funciona 
    (effect: Effect<number | Error>) => retry(effect, 2), 
    (effect: Effect<number | Error>) => orElse(effect, 0), 
    (effect: Effect<number>) => repeat(effect, 20), 
    timed, // como não precisa de outros argumentos, podemos passar diretamente
    (effect: Effect<[number[], number]>) => logEffect(effect, "Big Operation"), 
    run // o mesmo para o run
);

operation();

[--> Starting effect: Big Operation
Effect succeeded with value: 96
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 94
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 81
Effect succeeded with value: 78
Effect failed, retrying...
Effect succeeded with value: 74
Effect failed, retrying...
Effect succeeded with value: 90
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect failed, retrying...
Effect failed, retrying...
Effect failed, returning fallback value: 0
Effect succeeded with value: 92
Effect 

## Partial Application - Fazendo Melhor Ainda

Ainda conseguimos fazer melhor! O `pipe` é poderoso, mas temos um problema técnico.

### O Problema da Aridade

O `pipe` espera uma função que recebe **exatamente um argumento**, mas nossas funções recebem:
1. Um `Effect` como primeiro argumento  
2. Às vezes parâmetros de configuração (como `times`, `fallback`)

### Solução: Partial Application

**Técnica fundamental da programação funcional:** criar uma função que recebe alguns argumentos e retorna uma nova função que recebe o restante dos argumentos.

```typescript
// Função original com 2 argumentos
function add(a: number, b: number): number {
    return a + b;
}

// Partial application - "fixa" o primeiro argumento
const add5 = (b: number) => add(5, b);  // Equivale a: (b: number) => add(5, b)
const result = add5(3);  // result = 8
```

### Vantagens para Pipelines:

1. **Configuração Antecipada**: Definimos parâmetros uma vez
2. **Reutilização**: Mesma configuração para múltiplos casos
3. **Composição Limpa**: Todas as funções têm a mesma assinatura no pipeline
4. **Semântica Clara**: Nome da função indica claramente sua configuração

Com partial application, transformamos:
```typescript
(effect: Effect<A>) => retry(effect, 3)  // Verboso
```
Em:
```typescript
withRetry(3)  // Claro e reutilizável
```

In [37]:
function withRepeat<A>(times: number): (effect: Effect<A>) => Effect<A[]> {
    return (effect: Effect<A>) => repeat(effect, times);
}

// withRepeat(5)(randomNumberFunction()) == repeat(randomNumberFunction(), 5)

function withRetry<A>(times: number): (effect: Effect<A>) => Effect<A | Error> {
    return (effect: Effect<A>) => retry(effect, times);
}

function withOrElse<A>(fallback: A): (effect: Effect<A>) => Effect<A> {
    return (effect: Effect<A>) => orElse(effect, fallback);
}

function withTelemetry<A>(text: string): (effect: Effect<A>) => Effect<A> {
    return (effect: Effect<A>) => logEffect(effect, text);
}

const operation = pipe(
    randomNumberFunction, 
    withTelemetry("Random number generator"), 
    failIfSmallNumber, 
    withRetry(2), 
    withOrElse(0), 
    withRepeat(20), 
    timed, 
    run
);

operation()

// Você pode ler pensando:
// Rodar esse efeito com telemetria,
// se falhar, tentar mais duas vezes,
// se falhar, retornar 0,
// repetir 20 vezes,
// medir o tempo de execução

[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect succeeded with value: 98
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
[--> Starting effect: Random number generator
[--> End effect: Random number generator
Effect failed, retrying...
Effect failed, returning fallback value: 0
[--> Starting effect: Random number generator
[--> End effect: Rando

## Map e FlatMap - Transformações Poderosas

Duas das operações mais fundamentais em programação funcional. Elas permitem transformar valores **dentro** do contexto do Effect sem quebrar a composição.

### Map - Transformação Simples

É **LITERALMENTE** a mesma coisa que um map de array, mas para Effects:

```typescript
// Com arrays
const numbers = [1, 2, 3];
const strings = numbers.map(String);  // ["1", "2", "3"]

// Com Effects  
const numberEffect = () => 42;
const stringEffect = mapEffect(numberEffect, String);  // Effect<string>
```

**Importante**: Map mantém o contexto Effect. Se temos `Effect<A>` e aplicamos função `A -> B`, obtemos `Effect<B>`.

### FlatMap - Composição de Effects

E se a função que queremos aplicar também retorna um Effect? Teríamos `Effect<Effect<B>>` - um Effect aninhado!

FlatMap resolve isso "achatando" a estrutura:

```typescript
// Problema sem FlatMap
function duplicateEffect(x: number): Effect<number> {
    return () => x * 2;
}

const nested = mapEffect(numberEffect, duplicateEffect);  // Effect<Effect<number>> ❌

// Solução com FlatMap  
const flattened = flatMapEffect(numberEffect, duplicateEffect);  // Effect<number> ✅
```

### Casos de Uso Práticos:

- **Map**: Transformar dados (formatting, parsing, calculations)
- **FlatMap**: Composição de operações que podem falhar ou são assíncronas

In [74]:
function mapEffect<A, B>(effect: Effect<A>, func: (a: A) => B): Effect<B> {
    return () => {
        const result = effect();
        return func(result);
    };
}

// Isso permite usar funções diretamente
run(mapEffect(randomNumberFunction(), (x: number) => `Número aleatório: ${x}`));

function flatMapEffect<A, B>(effect: Effect<A>, func: (a: A) => Effect<B>): Effect<B> {
    return () => {
        const result = effect();
        return func(result)();
    };
}

function duplicateEffect(effect: Effect<number>): Effect<number> {
    return () => `Número duplicado: ${effect() * 2}`;
}

run(flatMapEffect(randomNumberFunction, duplicateEffect));

Número aleatório: 84
Número duplicado: 72


## Revisão Final - A Jornada dos Effects

Parabéns! 🎉 Você acabou de completar uma jornada profunda pelos **Effects** em programação funcional usando TypeScript. Vamos revisar o que construímos juntos:

### O que Aprendemos

#### 1. **Fundamentos dos Effects**
- **Thunks**: Funções que encapsulam computações para execução posterior
- **Lazy Evaluation**: Separação entre descrição e execução
- **Composição**: Como combinar Effects de forma elegante

#### 2. **Padrões de Resiliência**
- **Retry**: Tentativas automáticas em caso de falha
- **OrElse**: Graceful degradation com valores padrão
- **Error Handling**: Tratar erros como valores em vez de exceções

#### 3. **Observabilidade Transparente**
- **Logging**: Monitoramento sem modificar lógica de negócio
- **Telemetria**: Métricas de performance e timing
- **Debugging**: Instrumentação para troubleshooting

#### 4. **Evolução da Sintaxe**
- **Composição Aninhada**: O problema dos parênteses
- **Pipe Operator**: Melhoria na legibilidade
- **Partial Application**: Configuração reutilizável
- **Map/FlatMap**: Transformações funcionais poderosas

### Por que Isso Importa

**Effects não são apenas teoria acadêmica** - são uma ferramenta prática para construir sistemas:

- **Mais Robustos**: Retry e fallback automáticos
- **Mais Observáveis**: Telemetria transparente
- **Mais Testáveis**: Mocking e injeção de dependência simples
- **Mais Maintíveis**: Separação clara de responsabilidades

### Próximos Passos para Programação Funcional em TypeScript

#### 📚 **Conceitos Avançados para Estudar**

1. **Monads e Functors**
   - Maybe/Option para null safety
   - Either para error handling
   - IO Monad para side effects

2. **Functional Reactive Programming (FRP)**
   - RxJS streams
   - Event handling funcional
   - Observable patterns

3. **Immutability e Persistent Data Structures**
   - Immutable.js ou Immer
   - Structural sharing
   - Copy-on-write optimizations

4. **Advanced Composition Patterns**
   - Applicative Functors
   - Monad Transformers
   - Free Monads

#### 🛠️ **Ferramentas e Bibliotecas TypeScript**

**Programação Funcional:**
- `fp-ts` - Functional programming library
- `Ramda` - Functional utilities
- `lodash/fp` - Functional lodash
- `pipe-ts` - Lightweight pipe operator with excellent TypeScript support
- `Effect-TS` - Effect system completo

**Reactive Programming:**
- `RxJS` - Reactive Extensions for JavaScript
- `Most.js` - Ultra-high performance reactive programming

#### 🎯 **Projetos Práticos**

1. **CLI Tool Resiliente**
   - HTTP requests com retry
   - Configuration management
   - Error reporting

2. **Data Pipeline**
   - ETL com functional composition
   - Error handling e validação
   - Streaming de dados

3. **React App com FP**
   - State management funcional
   - Effect-based side effects
   - Composable components

#### 📖 **Recursos Recomendados**

**Livros:**
- "Functional-Light JavaScript" - Kyle Simpson
- "Professor Frisby's Mostly Adequate Guide to Functional Programming"
- "Category Theory for Programmers" - Bartosz Milewski

**Cursos:**
- "Functional Programming Principles in Scala" (Coursera)
- "Introduction to Functional Programming" (edX)

**Bibliotecas/Frameworks:**
- fp-ts documentation
- RxJS documentation
- Effect-TS guides

### Reflexão Final

Effects representam um **paradigm shift** fundamental: em vez de pensar em "como fazer", pensamos em "o que fazer" e deixamos o sistema decidir o "como".

Este mindset funcional não substitui a programação imperativa, mas a **complementa** de forma poderosa. Use Effects quando precisar de:

- **Composição complexa** de operações
- **Resiliência** automática
- **Testabilidade** high-level
- **Observabilidade** transparente

Continue experimentando, construindo e principalmente: **divirta-se** explorando esse mundo funcional! 🚀

---

*"The best way to learn functional programming is to write functional programs."* - **Anonymous FP Practitioner**